In [51]:
import numpy as np; import pandas as pd
import re; import datetime; from collections import Counter, OrderedDict
import category_encoders as ce
from sklearn.model_selection import train_test_split
# import pylogit as pl

In [52]:
df = pd.read_csv(r'가마우지\race.csv', engine='python', encoding='utf-8')

In [53]:
df['groupid'] = df['date'] + df['round'].apply(str)

In [54]:
idset = list(set(df['groupid']))
give_ids = dict(list(zip(idset, list(range(len(idset))))))

In [55]:
df['groupid'] = df['groupid'].apply(lambda x: give_ids[x])
df = df[df['distance'] > 0] # 거리 정보 있는 것만 살림

In [56]:
distances = list(set(df['distance']))
avg_vel = {}

In [57]:
for i in distances:
    avg_vel[i] = df[df['distance'] == i]['velocity'].mean()

In [58]:
df['avg_vel'] = df['distance'].apply(lambda x: avg_vel[x])
df['adj_vel'] = df['velocity'] / df['avg_vel']

In [59]:
df['prev1_rank']=df['prev1_rank'].fillna('0'); df['prev2_rank']=df['prev2_rank'].fillna('0')

In [60]:
df = df[(df['prev1_rank'] != 0) & (df['prev2_rank'] != 0)]

In [61]:
nums = list(map(str,range(1,17)))
df = df[(df['prev1_rank'].apply(lambda x: x in nums)) & (df['prev2_rank'].apply(lambda x: x in nums))]
df['prev1_rank'] = df['prev1_rank'].apply(int)
df['prev2_rank'] = df['prev2_rank'].apply(int)

In [62]:
df['prev_rank'] = df['prev1_rank'] + df['prev2_rank']
#del df['prev1_rank'], df['prev2_rank']

In [63]:
df['norating'] = (df['horse_level_num'] == 10).apply(int)

In [64]:
df = df[df['rank'].apply(lambda x: x in nums)]

In [65]:
df['rank'] = df['rank'].apply(int)

In [66]:
df['choice'] = (df['rank'] == 1).apply(int)

In [67]:
df['choice2'] = (df['rank'] < 3).apply(int)
df['choice3'] = (df['rank'] < 4).apply(int)

In [76]:
df['sex1'] = (df['sex'] == '수').apply(int)
df['sex2'] = (df['sex'] == '암').apply(int)

In [77]:
groupids = list(set(df['groupid']))
train, test = train_test_split(groupids, test_size = 0.3, random_state=datetime.datetime.now().second)

train = df[df['groupid'].apply(lambda x: x in train)]
test = df[df['groupid'].apply(lambda x: x in test)]

In [78]:
train2 = train[train['rank']>1]
train2['choice'] = (train2['rank'] == 2).apply(int)
train2['groupid'] = train2['groupid'] + 10000

train3 = train[train['rank']>2]
train3['choice'] = (train3['rank'] == 3).apply(int)
train3['groupid'] = train3['groupid'] + 20000

C:\Users\myeon\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\myeon\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\myeon\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try u

In [79]:
train = train.append(train2, ignore_index=True).append(train3, ignore_index=True)

In [72]:
train.columns

Index(['url', 'date', 'round', 'name', 'code', 'rank', 'lane', 'sex', 'age',
       'jockey_w', 'rating', 'jockey', 'difference', 'weight', 'dandivi',
       'yeondivi', 's1fr', 'c1r', 'c2r', 'c3r', 'c4r', 'g1fr', 's1f', 'c1',
       'c2', 'c3', 'c4', 'g3f', 'g1f', 'record', 'cure_in_1m', 'weight_diff',
       'raw_weight', 'weight_added', 'distance', 'velocity', 'prev1_rank',
       'prev2_rank', 'prev3_rank', 'prev1_velo', 'prev2_velo', 'prev3_velo',
       'horse_level_num', 'groupid', 'avg_vel', 'adj_vel', 'prev_rank',
       'norating', 'choice', 'choice2', 'choice3'],
      dtype='object')

In [99]:
del train['rank'], train['distance'], train['velocity']

basic_specification = OrderedDict()
basic_names = OrderedDict()
li = list(range(1,17))
xs = ['lane', 'sex1', 'sex2', 'age', 'jockey_w', 'dandivi', 'yeondivi', 'cure_in_1m', 'raw_weight', 'weight_added', 'horse_level_num', 'prev_rank', 'norating']
basic_specification["intercept"] = [li]
basic_names['intercept'] = ['annoying']

for i in xs:
    basic_specification[i] = [li]
    basic_names[i] = [i]

custom_alt_id = "lane"
obs_id_column = "groupid"
choice_column = "choice"

clogit_mnl = pl.create_choice_model(data=train_new.sort_values(by=['groupid']),
                                        alt_id_col=custom_alt_id,
                                        obs_id_col=obs_id_column,
                                        choice_col=choice_column,
                                        specification=basic_specification,
                                        model_type="MNL",
                                        names=basic_names)

clogit_mnl.fit_mle(np.ones(14))
clogit_mnl.get_statsmodels_summary()

In [119]:
train = train[train['horse_level_num'] > 0]

In [175]:
groupids = list(set(train['groupid']))

import random
tmps = []; memory = []

for i in groupids:
    dta = train[train['groupid']==i]
    diff = list(dta['lane'])
    dta = dta[dta['choice'] == 0]
    if len(dta)==0: 
        memory.append(i)
        continue
    for j in li:
        if j not in diff:
            r = random.randint(0, len(dta)-1)
            tmp=list(dta.iloc[r,:])
            tmp[0]=j
            tmps.append(tmp)

train_new = train.append(pd.DataFrame(tmps, columns = train.columns), ignore_index = True)

train_new = train_new[train_new['식별'] == False]

train_new['식별'] = train_new['groupid'].apply(lambda x: x in memory)

train_new

In [80]:
del train2, train3

In [82]:
train = train.sort_values(by=['groupid'])

In [83]:
test = test.sort_values(by=['groupid'])

In [85]:
train.columns

Index(['url', 'date', 'round', 'name', 'code', 'rank', 'lane', 'sex', 'age',
       'jockey_w', 'rating', 'jockey', 'difference', 'weight', 'dandivi',
       'yeondivi', 's1fr', 'c1r', 'c2r', 'c3r', 'c4r', 'g1fr', 's1f', 'c1',
       'c2', 'c3', 'c4', 'g3f', 'g1f', 'record', 'cure_in_1m', 'weight_diff',
       'raw_weight', 'weight_added', 'distance', 'velocity', 'prev1_rank',
       'prev2_rank', 'prev3_rank', 'prev1_velo', 'prev2_velo', 'prev3_velo',
       'horse_level_num', 'groupid', 'avg_vel', 'adj_vel', 'prev_rank',
       'norating', 'choice', 'choice2', 'choice3', 'sex1', 'sex2'],
      dtype='object')

In [89]:
test[use]

,choice,lane,age,jockey_w,dandivi,yeondivi,cure_in_1m,weight_diff,raw_weight,horse_level_num,adj_vel,prev_rank,norating,sex1,sex2,groupid
4705,0,6,6,52.0,44.7,9.9,1,1.0,528.0,5.0,0.988340,17,0,0,0,0
4703,0,8,4,52.5,10.3,2.6,0,-9.0,465.0,5.0,1.000850,9,0,0,1,0
4702,0,7,3,56.0,2.1,1.2,1,-5.0,437.0,5.0,1.000850,7,0,0,1,0
4701,0,2,4,53.0,6.7,1.9,0,5.0,470.0,5.0,1.010443,6,0,0,1,0
4700,0,3,5,52.0,23.4,3.6,1,0.0,400.0,5.0,1.013682,10,0,0,1,0
4698,1,1,4,52.0,3.8,1.4,1,-7.0,490.0,5.0,1.016941,14,0,0,1,0
4704,0,5,4,52.0,11.6,3.0,1,2.0,483.0,5.0,0.996122,14,0,0,1,0
46441,0,6,4,55.0,47.6,5.0,0,-4.0,405.0,5.0,0.988737,17,0,0,1,1
46442,0,7,3,54.0,25.0,2.4,0,-3.0,460.0,5.0,0.987686,14,0,0,1,1
46444,0,8,4,56.0,36.1,4.3,0,-1.0,482.0,5.0,0.982627,9,0,0,1,1


In [87]:
use = ['choice', 'lane', 'age', 'jockey_w', 'dandivi', 'yeondivi', 'cure_in_1m', 'weight_diff', 'raw_weight', 'horse_level_num', 'adj_vel', 'prev_rank', 'norating', 'sex1', 'sex2', 'groupid']

In [90]:
train[use].to_csv('clogit_train.csv', encoding='utf-8', index=False)
#train_new.to_csv('clogit_train_new.csv', encoding='utf-8', index=False)
test[use].to_csv('clogit_test.csv', encoding='utf-8', index=False)

In [135]:
df.to_csv('clogit_df.csv', encoding='utf-8', index=False)

# STATA 코드들
- train data만 넣은 뒤 -
clogit var1 var2 var3 var4 var5 var6 var7 var8 var9 var10 var11 var12 var14 var15, group(var16)
- test data append 하고 -
predict phat
- 이후 csv로 test data 부분의 phat column만 export한다 -> 빈 칸들 100으로 대체해줌.

In [107]:
why=list(pd.read_csv('dfdsaf.csv').iloc[:,0]) # STATA로 뽑은 phat

In [110]:
test['phat'] = why

In [116]:
test['delete']=test['phat'].apply(lambda x: x == 100)
test = test[test['delete'] == 0]

In [117]:
test['phat']=test['phat'].apply(float)

C:\Users\myeon\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [119]:
idx = list(set(test['groupid']))
dictt=dict(test.groupby(['groupid'])['phat'].max())
dandivi = []

t1=0; t2=0; t3=0
for i in idx:
    if list(test[(test['groupid']==i) & (test['phat']==dictt[i])]['choice'])[0] == 1:
        t1 = t1 + 1
        #dandivi.append(list(test[(test['groupid']==i) & (test['phat']==dictt[i])]['dandivi'])[0])
    if list(test[(test['groupid']==i) & (test['phat']==dictt[i])]['choice2'])[0] == 1:
        t2 = t2 + 1
    if list(test[(test['groupid']==i) & (test['phat']==dictt[i])]['choice3'])[0] == 1:
        t3 = t3 + 1 

In [85]:
import numpy as np
(np.array(dandivi) >= 3).mean()

0.292894280762565

In [127]:
total = test.groupby(['groupid']).max()['choice'].sum()
total2 = test.groupby(['groupid']).max()['choice2'].sum()
total3 = test.groupby(['groupid']).max()['choice3'].sum()

In [128]:
print(t1 / total)
print(t2 / total2)
print(t3 / total3)

0.6168582375478927
0.7588817705299942
0.805603201829617


In [124]:
for_accu=dict(test.groupby(['groupid']).count()['lane']<5) # 105 그룹 있음.

In [125]:
test['foraccu'] = test['groupid'].apply(lambda x: for_accu[x])

C:\Users\myeon\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [126]:
test_n = test[test['foraccu']==False]

In [133]:
idx = list(set(test_n['groupid']))
dictt=dict(test_n.groupby(['groupid'])['phat'].max())
dandivi = []

t1n=0; t2n=0; t3n=0
for i in idx:
    if list(test_n[(test_n['groupid']==i) & (test_n['phat']==dictt[i])]['choice'])[0] == 1:
        t1n = t1n + 1
        #dandivi.append(list(test[(test['groupid']==i) & (test['phat']==dictt[i])]['dandivi'])[0])
    if list(test_n[(test_n['groupid']==i) & (test_n['phat']==dictt[i])]['choice2'])[0] == 1:
        t2n = t2n + 1
    if list(test_n[(test_n['groupid']==i) & (test_n['phat']==dictt[i])]['choice3'])[0] == 1:
        t3n = t3n + 1 
        
totaln = test_n.groupby(['groupid']).max()['choice'].sum()
total2n = test_n.groupby(['groupid']).max()['choice2'].sum()
total3n = test_n.groupby(['groupid']).max()['choice3'].sum()

In [134]:
print(t1n / totaln)
print(t2n / total2n)
print(t3n / total3n)

0.6060606060606061
0.7512165450121655
0.802158273381295
